In [2]:
import pandas as pd
import numpy as np
import os
import simplejson


In [3]:
def load_file(path):
    file_path = path
    df = pd.read_csv(file_path)
    #Automatic fixes
    df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN
    return df


In [4]:
def file_params(df):
    params = {}
    #size
    params['size'] = {}
    params['size']['rows'] = int(df.shape[0])
    params['size']['cols'] = int(df.shape[1])

    #missing values
    params['missing'] = {}
    ##rows
    params['missing']['rows'] = int(df[df.isna().any(axis=1)].shape[0])
    params['missing']['rowsPercent'] = float(round(params['missing']['rows'] / params['size']['rows'], 7) * 100)
    ##cells
    params['missing']['cells'] = int(df.isna().sum().sum())
    params['missing']['cellsPercent'] = float(round(params['missing']['cells'] / (params['size']['rows'] * params['size']['cols']), 7) * 100)
    ##columns
    missingColumn = df.isna().sum()
    params['missing']['cols'] = missingColumn[missingColumn != 0].to_dict()
    ###percent of total values in colums
    nanByColumnPercent = round(df.isna().sum() / df.count().sum() * 100, 4)
    params['missing']['colsPercent'] = nanByColumnPercent[nanByColumnPercent !=0].to_dict()
    ###percent ot total missing values
    nanColumnContributionPercent = round(df.isna().sum() / df.isna().sum().sum() * 100, 2)
    params['missing']['colsPercentContribution'] = nanColumnContributionPercent[nanColumnContributionPercent !=0].to_dict()

    #names
    params['names'] = {}
    params['names']['cols'] = list(df.columns.values)
    params['names']['colsReverse'] = list(df.columns.values)
    params['names']['colsReverse'].reverse()

    #describe
    params['describe'] = df.describe().to_dict()

    return params

In [5]:
path =  ''
df_si = '123'
df2_si = '456'

df_sub1 = load_file('/Users/brandon/Desktop/test.csv')
df_sub1['storage_id'] = df_si
df_sub2 = load_file('/Users/brandon/Desktop/train.csv')
df_sub2['storage_id'] = df2_si

df = pd.concat([df_sub1, df_sub2])




In [21]:


grouped = df.groupby(df.storage_id)
df1 = grouped.get_group(df_si)

result = []

for storage_id in df['storage_id'].unique().tolist():
    result.append(grouped.get_group(storage_id))


result[0].drop('storage_id', axis=1, inplace=True)
result[0]

/var/folders/5z/fjwxc3_s5s1bbkthwhsjd5nm0000gn/T/ipykernel_87777/4160717868.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[0].drop('storage_id', axis=1, inplace=True)


,num,cat1,cat2,num_cont,target
0,8,A,one,a,True
1,9,B,two,0,False
2,6,A,one,1,True
3,3,B,two,7,False
4,1,A,one,10,True
5,5,A,two,2,False
6,10,A,one,1,True
7,9,A,two,5,False
8,6,B,one,0,True
9,3,C,two,8,False


['123', '456']

NameError: name 'result' is not defined